In [2]:
import pandas as pd
import numpy as np
import scipy as sp 
import seaborn as sns
from matplotlib import pyplot

%matplotlib inline

In [3]:
data = pd.read_csv("LoanStats3c.csv")

/usr/lib/python2.7/dist-packages/pandas/io/parsers.py:1070: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


# Data Cleaning

### Data description and corresponding usage

* id - keep (no analysis)

* member_id - keep (no analysis)

* loan_amnt - relevant

* funded_amnt  - relevant but nearly same as loan_amnt (see correlation matrix) - try to create a new feature

* funded_amnt_inv - almost same as funded_amnt ( can be removed)

* term - 36 or 60 - try to create a new feature or use another feature that takes in account this 'term' factor

* int_rate - relevant (can vary as a function of risk of the corresponding loan)

* installment - (very relevant)takes in account the term factor and loan_amnt together - we can remove 'term' column - but we should not remove loan_amnt because it gives the information about how big a loan is ? . 

* grade - LC determined value (grading scheme not clear) 

* sub_grade - LC determined value (grading scheme not clear)

* emp_title - Not relevant (since it does not gives us information about the financial condition of the borrower)

* emp_length - does provide information about the credibility of the borrower (needs cleaning - use regular expressions)

* home_ownership - relevant (provides information about the financial condition and other collateral the borrower has)

* annual_inc - very relevant (if the annual income of the borrower is not greater than the annual installments needs to be paid then its a bad loan)

* verification_status - relevant (we can provide a value corresponding to each factor or let the algorithm decide these values

* issue_d - irrelevant (does not provide any information about anything)

* loan_status - LABEL (Current loans needs to be removed)

* pymnt_plan - relevant(may add some information )

* url - irrelevant

* desc - relevant (may provide information about the intensions or behavior of the borrower) - Needs to remove NaN by some value

* purpose - relevant 

* title - same as purpose (remove either)

* zip_code - not relevant (can be replaced by the addr_state )

* dti - very relevant (debt to income ratio) - can think of removing income and loan_amnt for once

* delinq_2yrs - relevant (in terms of frequency)

* earlies_cr_line - relevant (can remove the month -year is relevant)

* inq_last_6mths - relevant inquiries ragarding loans in last 6 months 

* mths_since_last_delinq - relevant (need to deal with missing values)

* mths_since_last_record - relevant (need to deal with missing values)

* open_acc - relevant (open credit lines)

* pub_rec - relevant (negative factor)

* revol_bal - relevant (may be correlated to the amount of credit)

* total_acc - relevant (total number of credit lines currently in borrower's credit file )

* initial_list_status - relevant (initial listing status of loan ) (W,F -meaning not known till now)

* out_prncp - relevant 

* total_pymnt - relevant but correlated to out_prncp (see correlation)

* total_pymnt_inv - nearly same as total_pymnt (see similarity)

* total_rec_prncp - principal recieved till date (relevant)

* total_rec_late_fee - late fees recieved to date (relevance to be decided )

* total_rec_int - interest recieved till date

* recoveries - post charge off gross recovery (to be decided)

* collection_recovery_fee - post charge off collection fee () 

* last_pymnt_d - may be relevant

* last_pymnt_amnt - may be relevant

* next_pymnt_d - may be relevant

* last_credit_pull_d - The most recent month LC pulled credit for this loan ( may or may not be relevant)

* collections_12_mths_ex_med - Number of collections in 12 months excluding medical collections (relevant)

* mths_since_last_major_derog - Months since most recent 90-day or worse rating **find out the meaning** and then decide relevance

* policy_code - relevant (1 or 2) -decided according to the FICO scores

* application_type - individual or joint ( may be relevant)

* annual_inc_joint - relevant (but have lots of missing values for individual accounts)

* dti_joint - same case as above

* verification_status_joint - same as above

* acc_now_delinq - the number of accounts on which the borrower is now delinquent (relevant)

* tot_coll_amt - total collection amounts ever owed (relevant)

* tot_cur_bal - total current balance of all accounts (relevant) - try to make a new feature 

* open_acc_6m - Number of open trades in the last 6 months (many NaN values to deal with)

* open_il_6m - Number of currently active installment trades (lot of NaN values)

* open_il_12m - Number of installment accounts opened in past 12 months

* open_il_24m - Number of installment accounts opened in past 24 months

* mths_since_rcnt_il - months since most recent installment accounts openend 

* total_bal_il - total current balance of all installment accounts

* il_util - relevant (deal with NaNs) (ratio of total current balaance to high creditt/credit limit on all install acct

* open_rv_12m - Number of revolving trades in past 12 months (may or may not be )

* open_rv_24m - in past 24 months (Same)

* max_bal_bc - Maximum current balance owed on all revolving accounts (relevant) NaNs

* all_util - Balance to credit limit on all trades (relevant but NaNs)

* total_credit_rv - Total credit line on open revolving accounts (very relevant) -Try to create a new feature vector

* inq_fi - Number of personal finance inquiries (Lot of NaNs (scrutiny)

* total_fi_tl - Number of finance trades (may or may not be) (Lot of NaNs)

* inq_last_12m -Number of credit inquiries in past 12 months (Lot of NaNs) (scrutiny)

### First we need to remove the Current loans

In [5]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,
0,36805548,39558264,10400,10400,10400,36 months,6.99%,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000,Not Verified,Dec-2014,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,...
1,38098114,40860827,15000,15000,15000,60 months,12.39%,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000,Source Verified,Dec-2014,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,...
2,37612354,40375473,12800,12800,12800,60 months,17.14%,319.08,D,D4,Senior Sales Professional,10+ years,MORTGAGE,125000,Verified,Dec-2014,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,...
3,37822187,40585251,9600,9600,9600,36 months,13.66%,326.53,C,C3,Admin Specialist,10+ years,RENT,69000,Source Verified,Dec-2014,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,...
4,37842129,40605224,21425,21425,21425,60 months,15.59%,516.36,D,D1,Programming Analysis Supervisor,6 years,RENT,63800,Source Verified,Dec-2014,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,...


### Removing the Current loans

In [6]:
df = data[-(data.loan_status == 'Current')]

### Removing NaN values

In [7]:
df = df[-df.loan_status.isnull()]

In [8]:
df.loan_status

3             Fully Paid
5            Charged Off
6     Late (31-120 days)
11            Fully Paid
12            Fully Paid
17            Fully Paid
24            Fully Paid
28    Late (31-120 days)
29            Fully Paid
32            Fully Paid
35            Fully Paid
37            Fully Paid
40            Fully Paid
50            Fully Paid
55            Fully Paid
...
235599     Fully Paid
235600    Charged Off
235601     Fully Paid
235603     Fully Paid
235605     Fully Paid
235606     Fully Paid
235608     Fully Paid
235609     Fully Paid
235616     Fully Paid
235619     Fully Paid
235620    Charged Off
235621    Charged Off
235622     Fully Paid
235625    Charged Off
235626     Fully Paid
Name: loan_status, Length: 73292, dtype: object

In [9]:
sum(df.loan_status == 'In Grace Period')

1928

### Removing the "In Grace Period" Loans

These loans will not help us in classifying new loan as bad or good 

In [10]:
df = df[-(data.loan_status == 'In Grace Period')]

/usr/lib/python2.7/dist-packages/pandas/core/frame.py:1686: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [11]:
len(df.loan_status)

71364

### Giving Binary values to the loan_status variable which classifies them as good(1) or bad(0) loan

In [12]:
Label = []

for status in df.loan_status:
    if status == "Fully Paid":
        status = 1
        Label.append(status)
    else:
        status = 0
        Label.append(status)
        
        

In [13]:
df.loan_status = Label

In [14]:
df.loan_status

3     1
5     0
6     0
11    1
12    1
17    1
24    1
28    0
29    1
32    1
35    1
37    1
40    1
50    1
55    1
...
235599    1
235600    0
235601    1
235603    1
235605    1
235606    1
235608    1
235609    1
235616    1
235619    1
235620    0
235621    0
235622    1
235625    0
235626    1
Name: loan_status, Length: 71364, dtype: int64

In [15]:
sum(df.loan_status)

52023

In [16]:
df.corr()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,
member_id,1.000000,-0.026698,-0.026698,-0.026478,-0.040536,-0.008128,-0.024568,0.064463,-0.000418,-0.085196,0.016350,-0.049292,0.012721,-0.010274,0.019694,-0.006455,0.090916,0.090950,-0.082753,-0.082583,...
loan_amnt,-0.026698,1.000000,1.000000,0.999997,0.954410,0.410343,-0.061475,0.019603,-0.000564,-0.010466,-0.053376,0.022019,0.185368,-0.125125,0.351322,0.192695,0.184388,0.184386,0.770111,0.770103,...
funded_amnt,-0.026698,1.000000,1.000000,0.999997,0.954410,0.410343,-0.061475,0.019603,-0.000564,-0.010466,-0.053376,0.022019,0.185368,-0.125125,0.351322,0.192695,0.184388,0.184386,0.770111,0.770103,...
funded_amnt_inv,-0.026478,0.999997,0.999997,1.000000,0.954396,0.410374,-0.061480,0.019575,-0.000588,-0.010487,-0.053329,0.022066,0.185379,-0.125148,0.351305,0.192679,0.184419,0.184418,0.770093,0.770090,...
installment,-0.040536,0.954410,0.954410,0.954396,1.000000,0.407429,-0.049969,0.013022,0.011715,0.012371,-0.060426,0.005783,0.178358,-0.107900,0.341046,0.176189,0.155336,0.155335,0.748161,0.748142,...
annual_inc,-0.008128,0.410343,0.410343,0.410374,0.407429,1.000000,0.084076,-0.226094,0.052931,0.057332,-0.067830,-0.086484,0.146846,-0.036271,0.325084,0.204482,0.033185,0.033187,0.371695,0.371716,...
loan_status,-0.024568,-0.061475,-0.061475,-0.061480,-0.049969,0.084076,1.000000,-0.156588,-0.036069,-0.038670,0.045524,-0.032456,-0.013895,0.011811,0.008608,0.054324,-0.416765,-0.416757,0.481835,0.481849,...
dti,0.064463,0.019603,0.019603,0.019575,0.013022,-0.226094,-0.156588,1.000000,-0.011217,-0.000778,-0.004305,0.093337,0.275328,-0.073690,0.130162,0.192634,0.068848,0.068844,-0.062300,-0.062329,...
delinq_2yrs,-0.000418,-0.000564,-0.000564,-0.000588,0.011715,0.052931,-0.036069,-0.011217,1.000000,0.032516,-0.573496,-0.056361,0.048707,-0.017882,-0.038085,0.122631,0.027844,0.027839,-0.017764,-0.017795,...
inq_last_6mths,-0.085196,-0.010466,-0.010466,-0.010487,0.012371,0.057332,-0.038670,-0.000778,0.032516,1.000000,0.012876,-0.085696,0.102062,0.071866,-0.028749,0.137499,-0.002693,-0.002702,-0.020588,-0.020599,...


### Since correlation of funded amount and loan amount is 1.00000 we can remove one of them (we'll remove funded_amnt)

In [17]:
clean_data = df #so that we can keep the cleaned data as a back up

In [18]:
df.drop("funded_amnt",1,)

,id,member_id,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,
3,37822187,40585251,9600,9600,36 months,13.66%,326.53,C,C3,Admin Specialist,10+ years,RENT,69000,Source Verified,Dec-2014,1,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,...
5,37662224,40425321,7650,7650,36 months,13.66%,260.20,C,C3,Technical Specialist,< 1 year,RENT,50000,Source Verified,Dec-2014,0,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,...
6,37800722,40563521,12975,12975,36 months,17.86%,468.17,D,D5,Sales,10+ years,RENT,60000,Source Verified,Dec-2014,0,n,https://www.lendingclub.com/browse/loanDetail....,NaN,house,...
11,37741884,40504905,2500,2500,36 months,11.99%,83.03,B,B5,Manufacturing Engineer,< 1 year,MORTGAGE,89000,Source Verified,Dec-2014,1,n,https://www.lendingclub.com/browse/loanDetail....,NaN,home_improvement,...
12,37642222,40405288,5250,5250,36 months,11.44%,172.98,B,B4,Store Manager,2 years,RENT,26000,Not Verified,Dec-2014,1,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,...
17,37742142,40505198,2000,2000,36 months,14.99%,69.33,C,C5,practice plan associate,8 years,RENT,32200,Not Verified,Dec-2014,1,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,...
24,36109709,38821388,2000,2000,36 months,12.99%,67.38,C,C2,NaN,n/a,RENT,21312,Verified,Dec-2014,1,n,https://www.lendingclub.com/browse/loanDetail....,NaN,medical,...
28,37822030,40585070,18450,18450,36 months,14.31%,633.36,C,C4,construction foreman,10+ years,MORTGAGE,108000,Not Verified,Dec-2014,0,n,https://www.lendingclub.com/browse/loanDetail....,NaN,home_improvement,...
29,37692106,40465191,28000,28000,60 months,10.49%,601.70,B,B3,Humana Cares Manager,2 years,MORTGAGE,67000,Source Verified,Dec-2014,1,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,...
32,10089665,11941194,8000,8000,36 months,10.49%,259.99,B,B3,Senior Logistics Analyst,3 years,MORTGAGE,94000,Not Verified,Dec-2014,1,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,...
